### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from specmf.data import load_data
from specmf.models import Graph, MultiFidelityModel
from specmf.plot import (
    plot_cluster_size_hist,
    plot_data,
    plot_distributions,
    plot_loss_and_kappa,
    plot_mf_comparison,
    plot_spectrum,
)
from specmf.utils import error_analysis, load_model_config

In [ ]:
%%html
<style type='text/css'>
.CodeMirror{
font-size: 14px;
</style>

#### Data loading

In [ ]:
dataset_name = "beam"

x_LF, x_HF = load_data(
    dataset_name,
    preprocess=True,
    normalize=False, 
    flatten=True,
)

print(f"{x_LF.shape=}", f"{x_HF.shape=}")

### Plot data 

In [ ]:
plot_data(x_LF, x_HF, dataset_name, n_samples=3)

### Multi-fidelity model

#### Create graph and model instances

In [ ]:
# Create the graph
graph_config = {
    'metric': 'euclidean',
    'dist_space': 'ambient',
    'n_components': None,
    'method': 'full',
    'k_nn': None,
    'corr_scale': None,
    'k_adj': 7,
    'p': 0.5,
    'q': 0.5,
}
g_LF = Graph(data=x_LF, **graph_config)

#### Perform spectral clustering

In [ ]:
n_HF = 50
inds_train, labels = g_LF.cluster(n_HF)

In [ ]:
plot_cluster_size_hist(labels)

In [ ]:
# Plot Laplacian spectrum
eigvals, eigvecs = g_LF.laplacian_eig()
plot_spectrum(eigvals, 50)

#### Split high-fidelity data

In [ ]:
n_samples = x_HF.shape[0]
inds_test = np.delete(np.arange(n_samples), inds_train)
x_HF_train = x_HF[inds_train, :]
x_HF_test = x_HF[inds_test, :]

print(f"{x_HF_train.shape=}", f"{x_HF_test.shape=}")

#### Fit and train the model

In [ ]:
fit_model = True

# Create the model
model_config = {
    'sigma': 0.01,
    'method': 'full'
}
model = MultiFidelityModel(**model_config)

In [ ]:
if fit_model:
    x_MF, C_phi, dPhi, loss_history, kappa_history = model.fit_transform(
        g_LF,
        x_HF_train,
        inds_train,
        maxiter=20,
        verbose=False,
    )
    model.summary()
    plot_loss_and_kappa(loss_history, kappa_history)

else:
    from pathlib import Path

    notebook_path = Path.cwd()
    yaml_file_path = notebook_path.parent.parent / 'data/model_configs.yaml'
    model_config = load_model_config(yaml_file_path, dataset_name)

    model = MultiFidelityModel(**model_config)

    x_MF, C_phi, dPhi = model.transform(g_LF, x_HF_train, inds_train)
    model.summary()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.hist(dPhi, bins=20)
ax.set_xlabel("Variance", fontsize=14)
ax.set_ylabel("Frequency", fontsize=14)
ax.grid(True, linestyle="--", linewidth=0.5)
plt.tight_layout()
ax.tick_params(axis="both", labelsize=12)
ax.set_title("Variance histogram Beam data", fontsize=18)

### Results

In [ ]:
# Error Analysis for unseen test datadet
error_analysis(x_LF[inds_test], x_MF[inds_test], x_HF_test)

In [ ]:
# Error Analysis for the whole dataset
error_analysis(x_LF, x_MF, x_HF)

In [ ]:
E_LF = 100 * np.linalg.norm(x_LF - x_HF, axis=1) / (np.mean(np.linalg.norm(x_HF, axis=1)) + 1e-3)
E_MF = 100 * np.linalg.norm(x_MF - x_HF, axis=1) / (np.mean(np.linalg.norm(x_HF, axis=1)) + 1e-3)

plot_distributions(E_LF, E_MF, bins_LF=20, bins_MF=20, mask=None)

In [ ]:
x_values = np.linspace(0, 1, x_LF[0, :].shape[0])
xlabel = r"$x$"
ylabel = r"$w(x)$"
ymin, ymax = -4.5, 0

plot_mf_comparison(
    dataset_name=dataset_name,
    X_LF=x_LF,
    X_MF=x_MF,
    X_HF=x_HF,
    x_values=x_values,
    xlabel=xlabel,
    ylabel=ylabel,
    ymin=ymin,
    ymax=ymax,
    legend_loc="lower left",
)

In [ ]:
n_points, _ = x_LF.shape
fig, axs = plt.subplots(1, 3, figsize=(16, 4))
fig.subplots_adjust(wspace=0.15)
vmin, vmax = 1. * np.min((x_LF, x_HF)), 1.2 * np.max((x_LF, x_HF))
for i in range(100):
    j = np.random.randint(0, n_points)
    x_ = np.linspace(0, 1, x_LF[j, :].shape[0])
    axs[0].plot(x_, x_LF[j, :], label="Low-Fidelity")
    axs[0].set_title("Low-Fidelity")
    axs[1].plot(x_, x_HF[j, :], label="High-Fidelity")
    axs[1].set_title("High-Fidelity")
    axs[2].plot(x_, x_MF[j, :], label="Multi-Fidelity")
    axs[2].set_title("Multi-Fidelity")
    axs[0].set_ylim((vmin, vmax))
    axs[1].set_ylim((vmin, vmax))
    axs[2].set_ylim((vmin, vmax))

### Visualize uncertainty of multi-fidelity estimates

In [ ]:
import matplotlib.ticker as ticker


def fmt(x, pos):
    a, b = f'{x:.2e}'.split('e')
    b = int(b)
    return rf'${a} \times 10^{{{b}}}$'

In [ ]:
import umap


_, eigevs = g_LF.laplacian_eig()
X = np.real(eigvecs[:, :n_HF])


X_umap = umap.UMAP(
    n_components=2,
    n_neighbors=30,
    min_dist=1,
    init='random', 
    random_state=42
).fit_transform(X)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 9.5))
ax1 = ax.scatter(X_umap[:, 0], X_umap[:, 1], s=15, c=dPhi, vmax=0.045)
ax.scatter(X_umap[inds_train, 0], X_umap[inds_train, 1], s=30, c='r', marker='o')
cb = plt.colorbar(ax1, format=ticker.FuncFormatter(fmt))
cb.ax.tick_params(labelsize=16)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('UMAP of Cantilever Beam data (Case 4)', fontsize=26, pad=10)